[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/pdf.ipynb)

# PDF: Extrakce textu

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)


## Úvod a cíle kapitoly

V této kapitole si ukážeme základní principy práce s PDF soubory. Formát PDF zavedl v roce 1993 John Warnock, spoluzakladatel společnosti Adobe. Cílem bylo nalézt způsob, jak by se dokumenty z jakékoli aplikace daly jednoduše ukládat, posílat v elektronickém formátu a prohlížet a tisknout na jakémkoli počítači, aniž by došlo ke změně jejich podoby. 

PDF formát je nyní standardem pro digitalizaci knih či archivních dokumentů. V tomto kontextu rozlišujeme zejména mezi PDF soubory s rozpoznanou textovou vrstvou a bez ní. Digitalizovaný dokument bez rozpoznané textové vrstvi je víceméně pouze seznam obrázků. PDF s rozpoznanou textovou vrstvou má kromě vrstvy obrázků ještě vrstvu textových prvků, tzv. textových bloků. Textový blok je entita, která sestává z dat ohledně svého geometrického postavení na stránce (typicky dva body vymezující rohy obdelníku) a sestavy znaků textu. Jedná-li se o PDF dokument, který vznikl např. převodem `.docx` souboru, lze očekávat, že textový obsah bude bezchybný. Jedná-li se však o soubor, který vznikl digitalizací analogového dokumentu, často zde narazíme na určité nedostatky spjaté s technologií OCR. Této technologii se budeme věnovat v samostatné kapitole. 

V následujícím cvičení budeme PDF soubory zpracovávat pomocí Python knihovny `PyMuPDF`, která se do Python prostředí importuje pod přezdívkou `fitz`. 

Toto cvičení je postaveno na textech zpřístupněných na stránkách [scriptum.cz](https://scriptum.cz). Tato webová platforma zpřístupňuje českou exilovou a samizdatovou literaturu z období komunismu. Jedná se o projekt Sdružení občanů Exodus v Plzni a Třemošné, který funguje od roku 2007. Digitalizace textů je náplní práce lidí se zdravotním handicapem v rámci chráněné dílny. Jedná se o unikátní kolekci několika set titulů a více než 11 tisíc souborů. 

## Cvičení

In [ ]:
%%capture
!pip install PyMuPDF stanza wordcloud
import fitz
import stanza
stanza.download("cs")
nlp = stanza.Pipeline("cs")
from wordcloud import WordCloud
import requests
import io
import pickle
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
from bs4 import BeautifulSoup
import pandas as pd
import nltk


Pro ukázku si nyní do našeho Python prostředí načteme jedno číslo exilového časopisu *Studie*, revue Křesťanské akademie.  

In [ ]:
url = "https://scriptum.cz/soubory/scriptum/%5Bnode%5D/studie_1958_001_ocr_new.pdf"
pdf_object = io.BytesIO(requests.get(url).content)

In [ ]:
doc = fitz.open("pdf", pdf_object.read())

In [ ]:
doc.page_count

In [ ]:
p = doc.load_page(10)

In [ ]:
p

In [ ]:
pix = p.get_pixmap()

In [ ]:
pix.width

In [ ]:
pix.height

Příslušný obrázek stránky má v tuto chvíli podobu matice či tabulky o 585 sloupcích a 769 řádcích. Co se však nachází v jednotlivých buňkách? Abychom to mohli blíže prozkoumat, data si ještě jednou prozatimně převedeme do standardního maticového objektu (tzv. `array`) knihovny `numpy`. A následně se podíváme na malý výřez dat pro několik pixelů:

In [ ]:
np_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
np_array[400:404, 200:205]

Takto vypadají data pro celkově 20 pixelů. Jedná se o pixely z řádek 400 až 403 a sloupců 200 až 204 (pozor na indexování od nuly).

Trojice číslic udává pro každý pixel jeho barvu ve standardu RGB, o kterým si můžeme přečíst více např. na wikipedii [zde](https://cs.wikipedia.org/wiki/RGB). Každé číslo může nabývat na hodnotě 0-255 a jednotlivé hodnoty odpovídají intenzitám červené (*R*), zelené (*G*) a modré (*B*). Černá barva je definována hodnotami (0, 0, 0), zatímco bílá hodnotami (255, 255, 255). V případě že se jedná o obrázek, který pochází z textového dokumentu na bílém pozadí, můžeme očekvávat, že velké množství pixelů bude nabývat hodnot (255,255,255). Tam, kde se naopak nacházejí nulové hodnoty, bude se jednat o černou. Tam, kde jsou hodnoty pro všechny tři barvy stejné, půjde o barvu na škále šedi, od úplné černé až po bílou. 

To jsou důležité vlastnosti, na kterých je postaveno velké množství algoritmů pro zpracování obrázků, které mají například za cíl zvýšit jejich kontrast apod. To je klíčové i pro potřeby rozpoznávání znaků (OCR), kterému se budeme věnovat níže.

Nyní se však již podíváme na obrázek stránky jako takový. Můžeme jej vygenerovat přímo z maticových dat pomocí knihovny `matplotlib`:

In [ ]:
plt.imshow(np_array)

Snadno si zobrazíme např. pouze výřez této stránky.

In [ ]:
plt.imshow(np_array[410:730, 35:560])

Nyní se vraťme k našemu objektu `p`, který reprezentuje veškerá data spojená s danou stránkou. Velice přímočarým způsobem získáme kompletní textový obsah:

In [ ]:
print(p.get_text())

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="pdf", arg1=p.get_text())

Pokročilejší způsob představuje vyjmout textová data ze stránky po jednotlivých textových blocích, které obsahují i informaci o svém geometrickém umístění uvnitř stránky:

In [ ]:
p = doc.load_page(10)
textblocks = p.get_text_blocks() # ("blocks")
textblocks[:15]

Geometrie zde vymezuje dva body ohraničující obdelník, v němž se text nachází:

In [ ]:
rect = textblocks[2][:4]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(np_array)
patch = patches.Rectangle((rect[0], rect[1]),  # Bottom left corner
                          rect[2] - rect[0],  # Width
                          rect[3] - rect[1],  # Height
                          linewidth=1, edgecolor='r', facecolor='none')
ax.add_patch(patch)

Mít povědomí o těchto strukturálních vlastnostech PDF dokumentů je velice důležité, když z těchto dokumentů chceme získat strojově čitelný text pro další textové analýzy. PDF dokumenty jsou například často opatřeny záhlavím či zápatím, kde se objevuje třeba název periodika (jako je tomu zde), jméno autora, jméno příspěvku či číslo stránky. Vyextrahujeme-li ze všech stránek v daném dokumentu syrový text pomocí p.get_text(), budeme v našem textu mít i řetězce znaků z těchto textových bloků, což není žádoucí. Buď se tomu pokusíme předejít již při samotné extrakci, kdy můžeme využít geometrické polohy jednotlivých text bloků,  nebo se těchto dat pokusíme zbavit během následného čištění, např. pomocí *regulérních výrazů* (viz příslušná kapitola). Který případ je vhodnější závisí případ od případu a vyžaduje testování. Zde se budeme držet druhého případu.

In [ ]:
text = ""
for p in doc:
    text += p.get_text()

In [ ]:
text[5000:6000]

Zkusme se zbavit všech záhlaví pomocí regexu:

In [ ]:
pattern = "(_+)?(\n?Studie[\s_]+\d{4}\n?)(\W*\w{0,3}\s*/\s*\d\n?)?"
re.findall(pattern, text)[:10]

In [ ]:
text = re.sub(pattern, "\n", text)
text[3000:5000]

Na text aplikujeme ještě několik čistících funkcí.

In [ ]:
text = re.sub("\xad\n", "", text)
text = text.replace("- \n", "")
text = re.sub("\s\s+", " ", text.replace("\n", " "))
text[10000:11000]

Výsledek zdaleka není perfektní. V textu stále vidíme řadu problémů. Některé souvisejí s formátováním, jiné jsou dědictvím OCR analýzy. Pro naše aktuální potřeby však text v této podobě postačuje. 

Zkusme se však podívat na jiný text z téhož periodika:

In [ ]:
url = "https://scriptum.cz/soubory/scriptum/studie/studie_1978_055_ocr.pdf"
pdf_object = io.BytesIO(requests.get(url).content)
doc = fitz.open("pdf", pdf_object.read())
p = doc.load_page(20) # vybereme stránku
pix = p.get_pixmap()
np_array = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)


plt.imshow(np_array)

Vidíme, že text je nyní formátovaný odlišně. Například chybí původní záhlaví. Namísto toho vidíme v zápatí čísla stránek. V extrahovaném textu se pokusíme nyní zbavit i těch.

In [ ]:
p.get_text()

In [ ]:
re.sub("\n\s?\d{1,3}\s?\n", "\n", p.get_text())

In [ ]:
text = ""
for p in doc:
    text += p.get_text()

In [ ]:
pattern = "(_+)?(\n?Studie[\s_]+\d{4}\n?)(\W*\w{0,3}\s*/\s*\d\n?)?"
text = re.sub(pattern, "\n", text)
text = re.sub("\n\s?\d{1,3}\s?\n", "\n", text)
text = re.sub("\xad\n", "", text)
text = text.replace("- \n", "")
text = re.sub("\s\s+", " ", text.replace("\n", " "))
text[10000:12000]

Výsledný text má daleko od dokonalosti. My se s ním však nyní spokojíme a postoupíme dále. 



### Aplikace NLP na získaný text

In [ ]:
chunks = re.findall(r'.{0,50000}\.\s', text, re.DOTALL)
chunks_end = sum([len(chunk) for chunk in chunks])
chunks.append(text[chunks_end:])
docs = [nlp(chunk) for chunk in chunks]

In [ ]:
[len(chunk) for chunk in chunks]

In [ ]:
lemmatized_sents = []
for doc in docs:
    for sent in doc.sentences:
        lemmatized_sents.append([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]])

In [ ]:
lemmatized_sents[1000:1010]

In [ ]:
kontrola_pruchodu(ntb="pdf", arg1=lemmatized_sents[50])

> **Zde končí povinná část cvičení.** 

## Rozšiřující analýza 1: Extrakce textu ze všech čísel periodika

Nyní postoupíme dále a aplikujeme tento přístup na vyextrahování textu ze všech čísel daného periodika. Použijeme k tomu webovou stránku rozcestníku, na kterém jsou všechny soubory dostupné v uspořádaném seznamu:

https://scriptum.cz/soubory/scriptum/studie/

Použijeme obdobný postup, jaký jsme použili v kapitole o webscrapingu. Nejprve si vyextrahujeme seznam jmen všech relevantních souborů (zajímají nás pouze souboru, které končí "ocr.pdf" - i zde se nám hodí regex).

Tyto funkce jsou však již výpočetně poměrně náročné, tudíž tuto sekci je možné přeskočit.

In [ ]:
resp = requests.get("https://scriptum.cz/soubory/scriptum/studie/")
soup = BeautifulSoup(resp.content)
hrefs = [a.get('href') for a in soup.find_all("a")]
hrefs = [href for href in hrefs if re.search("ocr\.pdf$", href)]
hrefs = [href for href in hrefs if not ("rejstrik" in href or "obsah" in href)]
hrefs[:20]

In [ ]:
len(hrefs)

Nyní si vytvoříme funkci, do které vnoříme všechny extrakční, transformační a čistící procedury, které jsme prošli výše.  

In [ ]:
base_url = "https://scriptum.cz/soubory/scriptum/studie/"
def get_cleaned_text(filename):
    try:
        url = base_url + filename
        pdf_object = io.BytesIO(requests.get(url).content)
        doc = fitz.open("pdf", pdf_object.read())
        text = ""
        for p in doc:
            text += p.get_text()
        pattern = "(_+)?(\n?Studie[\s_]+\d{4}\n?)(\W*\w{0,3}\s*/\s*\d\n?)?"
        text = re.sub(pattern, "\n", text)
        text = re.sub("\n\s?\d{1,3}\s?\n", "\n", text)
        text = re.sub("\xad\n", "", text)
        text = re.sub("\s\s+", " ", text.replace("\n", " "))
        text = text.replace("- \n", "")
        year = int(re.search("\d{4}", filename).group())
        return filename, year, text
    except:
        pass

A následně tuto funkci aplikujeme jeden po druhém na všechny dostupné soubory pomocí cyklu FOR. Máme před sebou více než 100 jmen souborů, tj. vzneseme více než 100 HTTP dotazů. Tudíž provedení kódu zabere nějaký čas.

In [ ]:
%%time
scriptum_data = []
for filename in hrefs:
    filename, year, text = get_cleaned_text(filename)
    scriptum_data.append({"filename" : filename, "year" : year, "text" : text})

Vyextrahovaná data si převedeme do objektu typu `pandas.DataFrame` 

In [ ]:
scriptum_df = pd.DataFrame(scriptum_data)
scriptum_df.head(10)

Spočteme počet znaků v každém z námi vyextrahovaných textů a vytvoříme nový sloupec "n_chars", kam tuto hodnotu uložíme.

In [ ]:
scriptum_df["n_chars"] = scriptum_df["text"].str.len()
scriptum_df.head(5)

Díky tomu můžeme sečíst celkový počet znaků všech textů z daného periodika.

In [ ]:
scriptum_df["n_chars"].sum()

Tímto se nám tedy dostal do rukou další nemalý dataset zajímavých kulturních dat. Pokud pracujeme s repozitoří "pribehy-dat" jako celkem, dataset si uložíme do podsložky data:  

In [ ]:
scriptum_df.to_json("../data/scriptum_df.json")

## Rozšiřující analýza 2: Zpracování textových dat

Nyní trochu přeskočíme k tématu, kterým se budeme zabývat samostatně v jedné z jiných kapitol: kvantitativní textová analýza. Následující sérii kroků proto si proto v tuto chvíli nebudeme podrobně vysvětlovat, zaměříme se až na výsledná data.

Budeme k nim potřebovat knihovnu stanza a model pro předzpracování textových dat v češtině.

In [ ]:
scriptum_df = pd.read_json("../data/scriptum_df.json")

Pro testovací účely si vybereme jeden text z jednoho čísla:

In [ ]:
text = scriptum_df["text"].tolist()[0]
text[2000:3000]

Na celý dokument aplikujeme jazykový model pro předzpracování, který text automaticky:
* rozdělí do vět
* věty do slov
* jednotlivým slovům přiřadí *lemmata*, tj. převede je do tvarů, jak je najdeme ve slovníku (např. "je" -> "být").
* přiřadí jim "part-of-speech" (POS) tagy (např. "NOUN", "VERB" apod. "PUNCT" apod.)

In [ ]:
len(text)

In [ ]:
%%time
doc = nlp(text)

Vytvořili jsme nový `stanza` objekt `doc`, který obsahuje podrobně jazykově anotovanou reprezentaci celého textu. Z této reprezentace si nyní vyjmeme pouze lemmata vybraných slovních druhů: 

In [ ]:
lemmatized_sents = []
for sent in doc.sentences:
    lemmatized_sents.append([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]])
print(lemmatized_sents[100:110])

Ani v tomto případě nejsou výsledky ani zdaleka perfektní. Vše se odvíjí zejména z kvality vstupních dat. Vidíme např., že model si nedokáže poradit se slovy, které jsou ve stupních datech zachycena v rozdělené podobě apod. I přesto nyní postoupíme dále a aplikujeme danou proceduru na texty všech čísel. 

Opět si pro tento účel nadefinujeme speciální funkci.

In [ ]:
processed = []
!mkdir ../data/large_files
!mkdir ../data/large_files/lemsents
def get_lemmatized_sentences(filename, text, n_chars):
    if filename not in ["studie_1990_132_ocr.pdf"]:
        if filename + ".pickle" not in os.listdir("../data/large_files/lemsents/"):
            chunks = re.findall(r'.{0,50000}\.\s', text, re.DOTALL)
            chunks_end = sum([len(chunk) for chunk in chunks])
            chunks.append(text[chunks_end:])
            docs = [nlp(chunk) for chunk in chunks]
            lemmatized_sents = []
            for doc in docs:
                for sent in doc.sentences:
                    lemmatized_sents.append([t.lemma for t in sent.words if t.upos in ["PROPN", "NOUN", "VERB", "ADJ"]])
            processed.append(filename)
            pathfn = "../data/large_files/lemsents/" + filename + ".pickle"
            with open(pathfn, 'wb') as f:
                pickle.dump(lemmatized_sents, f)
            print(filename, n_chars, len(processed))
        else:
            pathfn = "../data/large_files/lemsents/" + filename + ".pickle"
            with open(pathfn, 'rb') as f:
                lemmatized_sents = pickle.load(f)
    else:
        lemmatized_sents = None
    return lemmatized_sents

Aplikace této funkce na všechny dokumenty v seznamu však zabere značný čas. Abychom se vyhnuli čekání, načteme si proto data, v kterých jsem již tuto proceduru aplikoval dříve (jak jsem to provedl je vidět v zakomentovaných příkazech v buňce níže.

In [ ]:
%%time
#scriptum_df["lemmatized_sents"] = scriptum_df.apply(lambda row: get_lemmatized_sentences(row["filename"], row["text"], row["n_chars"]), axis=1)
# scriptum_df.to_json("../data/large_files/scriptum_df_lemmata.json")
scriptum_df = pd.read_json("../data/large_files/scriptum_df_lemmata.json")
scriptum_df.head(5)

### Nejčastější slova po obdobích

In [ ]:
periods_freqs = {}
periods = [(1958,1968), (1969,1976), (1977,1990)]
periods_labels = ["Studie {0}-{1}".format(str(period[0]), str(period[1])) for period in periods]
for period, period_label in zip(periods, periods_labels):
    subset_df = scriptum_df[scriptum_df["year"].between(period[0], period[1])]
    lemmatized_sents = [sentences for sentences in subset_df["lemmatized_sents"] if sentences != None]
    sentences_flat = [sent for file_sents in lemmatized_sents for sent in file_sents]
    lemmata_list = [lemma for sent in sentences_flat for lemma in sent]
    lemmata_list = [lemma for lemma in lemmata_list if len(lemma) > 1]
    lemmata_freqs = nltk.FreqDist(lemmata_list).most_common()
    periods_freqs[period_label] = lemmata_freqs

Podívejme se namátkou na 50 nejčastějších slov z nejranějšího období.

In [ ]:
periods_freqs[periods_labels[0]][:50]

In [ ]:
wc = WordCloud(width=800, height=400).generate_from_frequencies(dict(periods_freqs[periods_labels[0]][:50]))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
n = 100
fig, axs = plt.subplots(3,1, figsize=(4.5, 5) , dpi=300, tight_layout=True)
for item, ax in zip(periods_freqs.items(), axs.ravel()):    
    wc = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(dict(item[1][:n]))
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(item[0])
    ax.axis("off")

### Lematizované věty s vybranými slovy

Nyní si nadefinujeme funkci, pomocí které budeme moci vyextrahovat veškeré věty obsahující konkrétní slova.

In [ ]:
def extract_target_sents(lemmatized_sents, targets):
    try:
        return [sent for sent in lemmatized_sents if any(target in sent for target in targets)]
    except:
        return []

In [ ]:
my_targets = ["kultura"]
scriptum_df["target_sents"] = scriptum_df["lemmatized_sents"].apply(extract_target_sents, args=(my_targets,))

In [ ]:
scriptum_df["target_sents"].apply(lambda x: len(x) if x != None else 0).sum()

In [ ]:
scriptum_df["target_sents"]

Jaká slova se v těchto větách objevují nejčastěji?

In [ ]:
column_sents = scriptum_df["target_sents"].tolist()
target_sents_counts = nltk.FreqDist([t for sent in [t for sent in column_sents for t in sent] for t in sent]) #
target_sents_counts = target_sents_counts.most_common()
target_sents_counts[:20]

In [ ]:
wc = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(dict(target_sents_counts[:50]))
plt.imshow(wc) # , interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
column_sents = scriptum_df["target_sents"].tolist()
target_sents_counts = nltk.FreqDist([t for sent in [t for sent in column_sents for t in sent] for t in sent]) #
target_sents_counts = target_sents_counts.most_common()
target_sents_counts[:20]